### ETL Australian user reviews

Este Archivo de trabajo, contiene las instrucciones necesarias para ejecutar la extración, transformación y carga del archivo "australian_user_reviews_.json".

La intención inicial del documento, es mostrar el paso a paso de la extracción de la información, explicar y justificar cada una de las instrucciones ejecutadas para la transformación del archivo y brindar como resultado un archivo limpio en formato CSV.

In [ ]:
# Se importan las librerías necesarias para trabajar con archivos de tipo json
import pandas as pd # Se utiliza pandas para trabajar con dataframes
import json # Se utiliza json para la lectura de archivos en este formato
import numpy as np # Se utiliza numpy para ciertas operaciones matematicas en los dataframe
from langdetect import detect # Langdetect es una biblioteca para detectar automáticamente el idioma en el que está escrito un texto
import nltk # NLTK (Natural Language Toolkit) es una plataforma para construir programas Python para trabajar con datos de lenguaje humano
from nltk.sentiment import SentimentIntensityAnalyzer # SentimentIntensityAnalyzer es una herramienta en NLTK para análisis de sentimientos

Haciendo una lectura previa del archivo, utilizando el editor de texto notepad++, podemos notar que el archivo no cumple con la estructura que debe de tener el formato json, es decir, que sus claves y valores, deben de venir especificados con comillas dobles, sin embargo, en el archivo están definidas con comillas simples.

Esto provoca que, tengamos que reemplazar estas comillas simples que delimitan las claves y valores, cuidando que no se sistituyan aquellas comillas simples existentes dentro de los valroes de tipo string como lo pueden ser las apostrofes.

In [ ]:
# En este apartado, comenzamos con la lectura del archivo json y colocamos su contenido en un dataframe de pandas
Ruta_Json = '..\\..\\Inputs Originales\\australian_user_reviews.json' # Guardamos la ruta del archivo json a leer en una variable

# Se prcede a leer el archivo json con la intención de reemplazar las comillas simples
with open(Ruta_Json, 'r', encoding='utf-8') as archivo: # Se crea la variable archivo para leer el archivo
    contenido = archivo.read() # Se guarda el resultado de lectura del archivo en una variable
    # Para la sititución de las comillas simples que rodean a las claves y valores, se hará utilizando ciertos patrones que justo delimitan las claves y valores
    contenido = contenido.replace("', '", '","').replace("{'", '{"').replace("': '", '": "').replace("':", '":').replace(", '", ', "')  # Se aplica la sistitución de los patrones encontrados

In [ ]:
# En este apartado, comenzamos con la lectura del archivo json y colocamos su contenido en un dataframe de pandas
Ruta_Json_Editado = '..\\..\\Inputs Originales\\australian_user_review_editado.json' # Guardamos la ruta del archivo json a leer en una variable

# Se procede a guardar el archivo editado como un nuevo archivo de tipo json
with open(Ruta_Json_Editado, 'w', encoding='utf-8') as archivo: # Se crea el archivo json editado
    archivo.write(contenido) # Se vacía el contenido dentro del archivo y se guarda

In [ ]:
# En este apartado, comenzamos con la lectura del archivo json editado y colocamos su contenido en un dataframe de pandas
Ruta_Json_Editado = '..\\..\\Inputs Originales\\australian_user_review_editado.json' # Guardamos la ruta del archivo json a leer en una variable

# Leemos el archivo de json editado
with open(Ruta_Json, 'r', encoding='utf-8') as archivo:
	datos = archivo.readlines()

registros = [eval(line.strip()) for line in datos]

# Se convierte el contenido trabajado previamente en un DataFrame
df_User_Review= pd.DataFrame(registros) # Se asigna el DataFrame resultante'

In [ ]:
# Ahora que tenemos un Dataframe con la información del archivo, comenzamos a explorar el contenido y realizamos ciertos trabajos de limpieza

# Analizamos la estructura y contenido del dataframe
df_User_Review.info() # Nos muestra la información general del dataframe
# Por ahora, tenemos un total de 5 columnas de tipo objeto

In [ ]:
# Se normaliza la columna de reviews
df_User_Review = pd.json_normalize(registros, record_path=['reviews'], meta=['user_id','user_url'] ) # Se guarda el resultado en el mismo Dataframe

In [ ]:
# haciendo un análisis estadístico del Dataframe, veremos cuántos registros contiene, el total de vacíos y duplicados

print("Total de filas en el DataFrame: ", df_User_Review.shape[0]) # Mostramos el total de registros del DataFrame 
# Se tiene un total de 88,146 registros dentro del archivo
print("Total de registros vacíos por columna: ", df_User_Review.isnull().sum()) # Mostramos el total de registros vacíos
# Se reemplazan los valores vacíos descritos como 'null', 'None' con NaN en todo el DataFrame
df_User_Review.replace(['', 'null', 'None'], np.nan, inplace=True) # Se guardel resultaddo en el DataFrame
# Debido a que vamos a analizar los reviews de los usuarios, eliminamos aquellos registros que no tienen review
df_User_Review = df_User_Review.loc[df_User_Review['review'].notna()] # Se eliminan los review vacios
# Se eliminan las columnas que estarían repetidas o pueden no ser útiles para nuestro análisis
df_User_Review = df_User_Review.drop(['funny','helpful','last_edited','user_url'], axis=1) # Se procede a eliminar aquellas columnas que no serían útiles para el análisis o que se pueden obtener desde otro csv
df_User_Review = df_User_Review.drop_duplicates(keep='first') # Se eliminan los registros duplicados
df_User_Review.sort_values('user_id') # Se ordena el dataframe por el ID

In [ ]:
# En este apartado se trabaja la columna "posted", para convertir sus valores en fecha y eliminar aquellos registros que no sean fiables para el análisis
df_User_Review['posted'] = df_User_Review['posted'].astype(str).str.replace("Posted ", '') # Se aplica la sistitución de la palabra posted por nada
df_User_Review['posted'] = df_User_Review['posted'].astype(str).str.replace(",", '') # Se aplica la sistitución de la coma por nada
df_User_Review['posted'] = df_User_Review['posted'].astype(str).str.replace(".", '') # Se aplica la sistitución de la coma por nada
df_User_Review['posted'] = pd.to_datetime(df_User_Review['posted'], errors='coerce') # Se convierten los valores obtenidos en un formato tipo fecha
df_User_Review = df_User_Review.loc[df_User_Review['posted'].notna()] # Se eliminan los registros vacios de la columna posted

In [ ]:
# En este apartado analizamos el idioma en el que están escritas las reseñas de los juegos

# Se define función para detectar el idioma según un texto
def detectar_idioma(texto): # Se recibe un texto string de parámetro
    try: # Se intentan capturar los errores
        return detect(texto) # Se retorna la detección del idioma
    except: # En caso de haber errores
        return None # No retorna nada la función

# Aplicamos la función de detección de idioma en una nueva columna llamada "lenguage"
df_User_Review['language'] = df_User_Review['review'].apply(detectar_idioma) # Se guarda el resultado en la columna "language"
df_User_Review = df_User_Review[df_User_Review['language'] == 'en'] # Se eliminan los registros que no sean en inglés
df_User_Review = df_User_Review.drop('language', axis=1) # Se borra la columna language

In [58]:
# A continuación, se procede a crear la columna de sentiment_analysis, la cual se ve apoyada de la columna review de nuestro data fram
# Esto con la intención de colocar tres valores principales de sentimiento, donde 0 es regular, 1 es neutral y 2 es bueno

# Se hace una limieza más intensa de nuestra columna review
df_User_Review.loc[:, 'review'] = df_User_Review['review'].str.lower() # Se coloca en minúsculas todas las letras del texto
df_User_Review['review'] = df_User_Review['review'].replace('[^A-Za-z0-9\s]+', '', regex=True) # Se eliminan los caracteres especiales del texto
df_User_Review.loc[:, 'review'] = df_User_Review['review'].str.replace('[^\w\s]', '', regex=True) # Se eliminan los caracteres que no sean alfanúmericos

In [59]:
# Se procede a crear la función para el analisis de sentimientos

# Se crea la función de análisis de sentimientos
def analyze_sentiments(df): # Se recibe como parámetro un dataframe
    
    sia = SentimentIntensityAnalyzer() # Se instancia el analizador de sentimientos

    # Se aplica el análisis de sentimientos y se asignan los valores a una nueva columna 
    df['score'] = df['review'].apply(lambda review: sia.polarity_scores(review)['compound'])
    # Se crea la columna de "Sentiment_analysis", se coloca valor 0 en caso de que el score previamente calculado sea 0, 1 en caso de que sea igual a 0 y en caso de que sea positivo será 2
    df['sentiment_analysis'] = df['score'].apply(lambda score: 0 if score < 0 else (1 if score == 0 else 2))

    # Se procede a crear una estadística con ciertos cálculos
    conteo_analisis = df['sentiment_analysis'].value_counts() # Se hace un conteo de los valores existentes en el análisis de sentimiento
    conteo_sin_review = df['review'].isnull().sum() # Se contabilizan aquellos registros vacios
    total_reviews = len(df) # Se asigna el total de registros recibidos

    # Calcular porcentajes
    conteo_analisis_porcentaje = (conteo_analisis / total_reviews * 100).round(2) # Se calcula el porcentaje de conteo de análisis
    conteo_sin_review_porncetaje = (conteo_analisis / total_reviews * 100).round(2) # Se calcula el porcentaje de reviews vacias
     
    # Se elimina la columna 'score', no necesitaremos estos datos
    df.drop(['score'], axis=1, inplace=True)  # Se aplica al mismo df recibido como argumento

    # Se retorna el dataframe utilizado y la estadística calculada
    return df, conteo_analisis, conteo_sin_review, conteo_analisis_porcentaje, conteo_sin_review_porncetaje

In [60]:
# Se llama a la función de analisis de sentimiento definida
# Se utiliza el dataframe que hemos trabajo más la creación de las variables que regresa la función de estadística
df_User_Review, conteo, reviews_vacias, conteo_porncentaje, reviews_porncentaje = analyze_sentiments(df_User_Review)

# Crear un nuevo DataFrame con el conteo y porcentaje
resumen_sentimientos = pd.DataFrame({ # Se define el nuevo dataframe
    'Conteo': conteo, # Se agrega la columna conteo
    'Porcentaje': conteo_porncentaje.round(2).astype(str) + '%' # Se agrega la columna porcentaje
})

# Ordenar el DataFrame por el conteo de mayor a menor
resumen_sentimientos = resumen_sentimientos.sort_values(by='Conteo', ascending=False)

In [61]:
# Debido a que nuestra temporalidad de análisis será anual, se procede a modificar la columna posted para solo obtener el año
df_User_Review['year'] = df_User_Review['posted'].dt.year.astype('Int64')
df_User_Review = df_User_Review.drop('posted', axis=1) # Se elimina la columna de posted

In [62]:
# Se guarda el resultado de la limpieza en un nuevo archivo de tipo csv
df_User_Review.to_csv('..\\..\\Archivos Limpios\\australian_user_review_limpio.csv', index=False)

# Por ahora, se ha terminado con la limpieza de la información del archivo user_reviews